#  Todo
* How do we keep an option to hover on the countries that have no data?
* How do we hide the countries from the map that are *never* a part of the datasets? (Russia, Iceland etc)
* How do we make a slider work?
* Should we do a "dias show"? In that case: How?
* Which datasets should we work with and which points do we want to make?

In [65]:
import pandas as pd

df = pd.read_csv("drugs.csv", sep=";")
df.head(3)

,Country,2018
0,Austria,42610.0
1,Belgium,51774.0
2,Bulgaria,2433.0


In [66]:
empty_df = df[df['2018'].isnull()]
empty_df["2018"] = [0] * len(empty_df)
empty_df

<ipython-input-66-34ac0e54179f>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Country,2018
5,Czechia,0
25,Slovenia,0


In [67]:
import plotly.express as px
import plotly.graph_objects as go

#df = px.data.gapminder().query("year==2007")


fig = px.choropleth(df, locations="Country",
                    color="2018", # lifeExp is a column of gapminder
                    hover_name="Country", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Plasma,
                    scope="europe",
                    locationmode="country names",
                    custom_data=["Country", "2018"])


fig2 = px.choropleth(empty_df, locations="Country",
                    color="2018", # lifeExp is a column of gapminder
                    hover_name="Country", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Plasma,
                    scope="europe",
                    locationmode="country names",
                    custom_data=["Country", "2018"])

fig.update_traces(hovertemplate="<b>Country: %{customdata[0]}<br></b><br>" +
                                     "Drug Offenders: %{customdata[1]:n}<br>", selector=dict(type='choropleth'))
fig2.show()
fig.show()